# Convert measured experimental compositions to endmember fraction

In [ ]:
%load_ext snakeviz

In [ ]:
import thermoengine as thermo

%run "core.ipynb"
analysis = load_analysis()


exp_data = analysis['exp_data']
phs_data = analysis['phs_data']
phs_mol_comp = analysis['phs_mol_comp']
phs_wt_comp = analysis['phs_wt_comp']

In [ ]:
modelDB = thermo.model.Database()

In [ ]:
#Selects the experiment indices that are not found in the Liquid tab; this is problematic
#because some experiments show liquid present in the phases column but don't report a liquid
#and others report no liquid in the phases column but report a liquid composition; indicies 
#are effed up...

exp_sans_liq = []
for idx in exp_data.index:
    if idx not in phs_wt_comp['Liquid'].index:
        exp_sans_liq.append(idx)
    else:
        pass

In [ ]:
#NOTE do not use this
phase_symbols = []
phase_names = []
phase_wt_oxide_comp = []
P=[]
T=[]

for idx in exp_sans_liq:
    P.append(exp_data.loc[idx, 'P (GPa)'])
    T.append(exp_data.loc[idx, 'T (C)'])
    
    for key in phs_wt_comp:
        iphs_wt_comp = phs_wt_comp[key]
        if idx in iphs_wt_comp.index:
            phase_names.append(key)
            phase_symbols.append(LEPR_phase_symbols[key])
            phase_wt_oxide_comp.append(iphs_wt_comp.loc[idx].values)
        
phase_wt_oxide_comp = np.array(phase_wt_oxide_comp)
phase_stability_data = thermo.calibrate.PhaseStabilityExp(iP, iT, 0, phase_symbols, phase_wt_oxide_comp, modelDB)

    

In [ ]:
phase_wt_oxide_comp


In [ ]:
#exp_idx=337
#exp_idx = 1350
phase_stability_exps=[]

for exp_idx in exp_data.index:
    iP = exp_data.loc[exp_idx, 'P (GPa)']
    iT = exp_data.loc[exp_idx, 'T (C)']

# [phs_wt_comp[key].loc[exp_idx] for key in phs_wt_comp ]

    phase_symbols = []
    phase_names = []
    phase_wt_oxide_comp = []

    for key in phs_wt_comp:
        iphs_wt_comp = phs_wt_comp[key]
        if exp_idx in iphs_wt_comp.index:
            phase_names.append(key)
            phase_symbols.append(LEPR_phase_symbols[key])
            phase_wt_oxide_comp.append(iphs_wt_comp.loc[exp_idx].values)
        
    phase_wt_oxide_comp = np.array(phase_wt_oxide_comp)
    phase_stability_exps = thermo.calibrate.PhaseStabilityExp(iP, iT, 0, phase_symbols, phase_wt_oxide_comp, modelDB)
    print(phase_stability_data)

#run on subset of exps without Liq

In [ ]:
display(phase_names)
display(phase_symbols)
display(phase_wt_oxide_comp)

In [ ]:
phase_stability_data = thermo.calibrate.PhaseStabilityExp(iP, iT, 0, phase_symbols, phase_wt_oxide_comp, modelDB)


In [ ]:
phase_stability_data.calc_equil_rxn_affinities()

In [ ]:
phase_stability_data._phase_mol_endmem_comp

In [ ]:
phase_mol_oxide_comp=thermo.chem.wt_to_mol_oxide(phase_wt_oxide_comp)

In [ ]:
rel_phases=[]
for iphs in phase_symbols:
    rel_phases.append(modelDB.get_phase(iphs))

rel_phases

In [ ]:
phase_mol_endmem_comp = {}
for phs_sym, phase, mol_oxide_comp in zip(
    phase_symbols, rel_phases, phase_mol_oxide_comp):

    if phs_sym not in modelDB.phase_obj['pure']:
        endmem_comp = phase.calc_endmember_comp(
            mol_oxide_comp, method='intrinsic', output_residual=False)
    else:
        endmem_comp = 1

    phase_mol_endmem_comp[phs_sym] = np.array(endmem_comp)

In [ ]:
phase_mol_endmem_comp

# Following code get's set of valid reactions for one experiment

In [ ]:
rxn_svd_props = thermo.chem.calc_reaction_svd(phase_symbols, TOLsvd=1e-4, modelDB=modelDB)
rxn_svd = rxn_svd_props['rxn_svd']
Nbasis=len(rxn_svd)
rxn_endmember_name = rxn_svd_props['all_endmember_name']
rxn_phase_symbols = rxn_svd_props['all_phase_symbol']

In [ ]:
endmember_ids = rxn_svd_props['all_endmember_id']
rxns = []
for irxn_coefs in rxn_svd:
    irxn = modelDB.get_rxn(rxn_phase_symbols, endmember_ids, irxn_coefs)
    
    rxns.append(irxn)

In [ ]:
rxns

In [ ]:
rxn_svd

In [ ]:
rxn_endmember_name

In [ ]:
wtcoefs, costs, rxn_coefs, wtcoefs_ortho = thermo.chem.get_rxns(rxn_svd, Ndraw=3, ortho_scale=15, Nbasis=Nbasis, TOL=1e-10)

In [ ]:
(np.place(rxn_coefs, abs(rxn_coefs)< 1e-2, 0))

In [ ]:
endmember_ids = rxn_svd_props['all_endmember_id']
rxns = []
for irxn_coefs in rxn_coefs:
    irxn = modelDB.get_rxn(rxn_phase_symbols, endmember_ids, irxn_coefs)
    
    rxns.append(irxn)

In [ ]:
phases_cls = thermo.phases.Rxn

In [ ]:
affinities=[]

for imol_comp in phase_mol_endmem_comp:
    affinities.append(phases_cls.affinity(iT, iP, imol_comp))
    
affinities

In [ ]:
iphs_name = 'Garnet'
iphs = modelDB.get_phase('Grt')
iendmem_names = iphs.endmember_names

imol_comp = phs_mol_comp[iphs_name]

In [ ]:
iendmem_comp = []

for idx in imol_comp.T:
    ijmol_comp = imol_comp.loc[idx].values
    
    ijendmem_comp = iphs.calc_endmember_comp(
        ijmol_comp, output_residual=False)
    iendmem_comp.append(ijendmem_comp)
    
iendmem_comp = np.array(iendmem_comp)
    

In [ ]:
# NOTE some of these values are NAN for some reason?
iendmem_comp[80:100,:]

In [ ]:
plt.figure()
plt.imshow(iendmem_comp.T[:,:50], cmap='magma', vmin=0, vmax=1)
plt.colorbar(orientation='horizontal', label='Endmem Frac')
ax = plt.gca()
ax.set_yticks([0,1,2])
ax.set_yticklabels(iendmem_names);


### Garnet composition space

### This code is old code that likely will no longer be used; it originally filtered phase assemblages from the garnet excel file for only unique phase assemblages; we are no longer implementing this idea

In [ ]:
unique_phase_symbols = ['Cpx', 'Ol', 'Grt']
rxn_svd_props = thermo.chem.calc_reaction_svd(unique_phase_symbols, TOLsvd=1e-4, modelDB=modelDB)
rxn_svd = rxn_svd_props['rxn_svd']
Nbasis=len(rxn_svd)
rxn_endmember_name = rxn_svd_props['all_endmember_name']
rxn_phase_symbols = rxn_svd_props['all_phase_symbol']

In [ ]:
unique_phase_assemblages = rxn_data['Experiment']['Phases'].unique()

In [ ]:
phase_assemblages=[]
for iphase_assem in unique_phase_assemblages:
    
    try: 
        phase_list = iphase_assem.split(sep='+')
        phase_list.sort()
        phase_assemblages.append(phase_list)
        
    except: 
        print('empty_assemblage')
        
phase_assemblages = np.unique(phase_assemblages)

In [ ]:
mask = []
for iassemblage in phase_assemblages:
    imask = 'liq' not in iassemblage
    
    mask.append(imask)
    
mask = np.array(mask)
np.sum(mask)

In [ ]:
phase_assemblages[mask]

In [ ]:
wtcoefs, costs, rxn_coefs, wtcoefs_ortho = thermo.chem.get_rxns(rxn_svd, Ndraw=3, ortho_scale=15, Nbasis=Nbasis, TOL=1e-10)

In [ ]:
(np.place(rxn_coefs, abs(rxn_coefs)< 1e-2, 0))

In [ ]:
endmember_ids = rxn_svd_props['all_endmember_id']
rxns = []
for irxn_coefs in rxn_coefs:
    irxn = modelDB.get_rxn(rxn_phase_symbols, endmember_ids, irxn_coefs)
    
    rxns.append(irxn)  

In [ ]:
phases = []

for irxn in rxns:
    phases.extend(irxn.phases)
    
phases = np.unique(phases)